# Futsal Court Analytics - Exploratory Data Analysis
## 6-Month Performance Analysis for 2 Courts

**Confidential Business Data**

This notebook performs comprehensive EDA on futsal court booking data to:
- Analyze utilization patterns
- Identify revenue opportunities
- Understand customer behavior
- Support business decisions

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries loaded successfully")

## 1. Data Generation (Mock Data)
### Replace this section with your actual data loading when available

In [ ]:
# Generate mock data for 6 months
np.random.seed(42)

# Date range: 6 months of data
start_date = datetime.now() - timedelta(days=180)
dates = pd.date_range(start=start_date, periods=180, freq='D')

# Time slots: 8 AM to 11 PM (15 hours = 15 slots)
time_slots = [f"{h:02d}:00" for h in range(8, 23)]

# Generate booking data
data = []

for date in dates:
    day_of_week = date.dayofweek
    is_weekend = day_of_week >= 5
    
    for court in ['Court 1', 'Court 2']:
        for time_slot in time_slots:
            hour = int(time_slot.split(':')[0])
            
            # Peak hours: 6 PM - 10 PM have higher booking rates
            is_peak = 18 <= hour <= 22
            
            # Base booking probability
            base_prob = 0.45 if is_weekend else 0.35
            if is_peak:
                base_prob += 0.30
            
            # Court 2 is slightly less popular
            if court == 'Court 2':
                base_prob *= 0.85
            
            # Simulate booking
            is_booked = np.random.random() < base_prob
            
            if is_booked:
                # Generate revenue (varies by time)
                base_price = 50 if is_peak else 40
                revenue = base_price + np.random.normal(0, 5)
                
                # Customer type
                customer_type = np.random.choice(
                    ['Regular', 'One-time', 'Corporate'],
                    p=[0.55, 0.35, 0.10]
                )
                
                # Cancellation/No-show
                cancellation_prob = 0.08 if customer_type == 'One-time' else 0.03
                was_cancelled = np.random.random() < cancellation_prob
                
                no_show_prob = 0.05 if customer_type == 'One-time' else 0.02
                was_no_show = np.random.random() < no_show_prob if not was_cancelled else False
                
                # Booking advance days
                if customer_type == 'Regular':
                    advance_days = np.random.choice([1, 2, 3, 7], p=[0.2, 0.3, 0.3, 0.2])
                elif customer_type == 'Corporate':
                    advance_days = np.random.choice([7, 14, 21], p=[0.5, 0.3, 0.2])
                else:
                    advance_days = np.random.choice([0, 1, 2], p=[0.4, 0.4, 0.2])
                
                data.append({
                    'date': date,
                    'court': court,
                    'time_slot': time_slot,
                    'hour': hour,
                    'day_of_week': date.strftime('%A'),
                    'is_weekend': is_weekend,
                    'is_peak_hour': is_peak,
                    'is_booked': True,
                    'revenue': revenue,
                    'customer_type': customer_type,
                    'was_cancelled': was_cancelled,
                    'was_no_show': was_no_show,
                    'advance_booking_days': advance_days,
                    'actual_revenue': 0 if (was_cancelled or was_no_show) else revenue
                })
            else:
                # Empty slot
                data.append({
                    'date': date,
                    'court': court,
                    'time_slot': time_slot,
                    'hour': hour,
                    'day_of_week': date.strftime('%A'),
                    'is_weekend': is_weekend,
                    'is_peak_hour': is_peak,
                    'is_booked': False,
                    'revenue': 0,
                    'customer_type': None,
                    'was_cancelled': False,
                    'was_no_show': False,
                    'advance_booking_days': None,
                    'actual_revenue': 0
                })

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
df['year_month'] = df['date'].dt.to_period('M')
df['week'] = df['date'].dt.isocalendar().week

print(f"✅ Generated {len(df):,} records")
print(f"📅 Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"🏟️  Courts: {df['court'].nunique()}")
print(f"📊 Total bookings: {df['is_booked'].sum():,}")

## 2. Data Overview & Quality Check

In [ ]:
# Display first few rows
print("📋 Sample Data:")
display(df.head(10))

# Data info
print("\n📊 Dataset Information:")
df.info()

In [ ]:
# Check for missing values
print("🔍 Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
display(missing_df[missing_df['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("✅ No missing values found!")

In [ ]:
# Summary statistics
print("📈 Summary Statistics:")
display(df.describe())

## 3. Court Utilization Metrics

In [ ]:
# Overall utilization rate
total_slots = len(df)
booked_slots = df['is_booked'].sum()
utilization_rate = (booked_slots / total_slots) * 100

print("🏟️  OVERALL COURT UTILIZATION")
print(f"Total Available Slots: {total_slots:,}")
print(f"Booked Slots: {booked_slots:,}")
print(f"Utilization Rate: {utilization_rate:.2f}%")
print(f"Empty Slots: {total_slots - booked_slots:,}")

In [ ]:
# Utilization by court
court_util = df.groupby('court').agg({
    'is_booked': ['count', 'sum', lambda x: (x.sum() / len(x)) * 100]
}).round(2)
court_util.columns = ['Total Slots', 'Booked Slots', 'Utilization %']

print("\n📊 Utilization by Court:")
display(court_util)

In [ ]:
# Visualize court comparison
fig = go.Figure()

for court in df['court'].unique():
    court_data = df[df['court'] == court]
    monthly = court_data.groupby('year_month')['is_booked'].apply(
        lambda x: (x.sum() / len(x)) * 100
    )
    
    fig.add_trace(go.Scatter(
        x=monthly.index.astype(str),
        y=monthly.values,
        mode='lines+markers',
        name=court,
        line=dict(width=3)
    ))

fig.update_layout(
    title='Monthly Utilization Rate by Court',
    xaxis_title='Month',
    yaxis_title='Utilization Rate (%)',
    hovermode='x unified',
    height=500
)

fig.show()

## 4. Peak Hours & Days Analysis

In [ ]:
# Utilization by hour
hourly_util = df.groupby('hour').agg({
    'is_booked': lambda x: (x.sum() / len(x)) * 100
}).round(2)
hourly_util.columns = ['Utilization %']

print("⏰ Hourly Utilization Patterns:")
display(hourly_util.sort_values('Utilization %', ascending=False))

In [ ]:
# Heatmap: Hour vs Day of Week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = df.groupby(['day_of_week', 'hour'])['is_booked'].apply(
    lambda x: (x.sum() / len(x)) * 100
).reset_index()
heatmap_pivot = heatmap_data.pivot(index='day_of_week', columns='hour', values='is_booked')
heatmap_pivot = heatmap_pivot.reindex(day_order)

fig = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    colorscale='RdYlGn',
    text=heatmap_pivot.values.round(1),
    texttemplate='%{text}%',
    textfont={"size": 10},
    colorbar=dict(title="Utilization %")
))

fig.update_layout(
    title='Utilization Heatmap: Day of Week vs Hour',
    xaxis_title='Hour of Day',
    yaxis_title='Day of Week',
    height=500
)

fig.show()

In [ ]:
# Weekend vs Weekday comparison
day_type_util = df.groupby('is_weekend')['is_booked'].apply(
    lambda x: (x.sum() / len(x)) * 100
).round(2)

fig = go.Figure(data=[
    go.Bar(
        x=['Weekday', 'Weekend'],
        y=[day_type_util[False], day_type_util[True]],
        text=[f"{day_type_util[False]:.1f}%", f"{day_type_util[True]:.1f}%"],
        textposition='auto',
        marker_color=['#3498db', '#e74c3c']
    )
])

fig.update_layout(
    title='Utilization: Weekday vs Weekend',
    yaxis_title='Utilization Rate (%)',
    height=400
)

fig.show()

print(f"\n📊 Weekend bookings are {day_type_util[True] - day_type_util[False]:.1f}% higher than weekdays")

## 5. Revenue Analysis

In [ ]:
# Overall revenue metrics
booked_df = df[df['is_booked'] == True]

total_potential_revenue = booked_df['revenue'].sum()
total_actual_revenue = booked_df['actual_revenue'].sum()
revenue_loss = total_potential_revenue - total_actual_revenue
revenue_loss_pct = (revenue_loss / total_potential_revenue) * 100

print("💰 REVENUE OVERVIEW")
print(f"Total Potential Revenue: ${total_potential_revenue:,.2f}")
print(f"Total Actual Revenue: ${total_actual_revenue:,.2f}")
print(f"Revenue Loss (Cancellations/No-shows): ${revenue_loss:,.2f} ({revenue_loss_pct:.2f}%)")
print(f"\nAverage Revenue per Booking: ${booked_df['revenue'].mean():.2f}")
print(f"Average Actual Revenue per Booking: ${booked_df['actual_revenue'].mean():.2f}")

In [ ]:
# Revenue by court
court_revenue = booked_df.groupby('court').agg({
    'revenue': 'sum',
    'actual_revenue': 'sum',
    'is_booked': 'count'
}).round(2)
court_revenue.columns = ['Potential Revenue', 'Actual Revenue', 'Total Bookings']
court_revenue['Revenue per Booking'] = (court_revenue['Actual Revenue'] / court_revenue['Total Bookings']).round(2)

print("\n📊 Revenue by Court:")
display(court_revenue)

In [ ]:
# Monthly revenue trend
monthly_revenue = booked_df.groupby('year_month').agg({
    'revenue': 'sum',
    'actual_revenue': 'sum'
}).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly_revenue['year_month'].astype(str),
    y=monthly_revenue['revenue'],
    name='Potential Revenue',
    marker_color='lightblue'
))

fig.add_trace(go.Bar(
    x=monthly_revenue['year_month'].astype(str),
    y=monthly_revenue['actual_revenue'],
    name='Actual Revenue',
    marker_color='darkblue'
))

fig.update_layout(
    title='Monthly Revenue: Potential vs Actual',
    xaxis_title='Month',
    yaxis_title='Revenue ($)',
    barmode='group',
    height=500
)

fig.show()

In [ ]:
# Revenue by hour (to identify pricing opportunities)
hourly_revenue = booked_df.groupby('hour').agg({
    'actual_revenue': ['sum', 'mean', 'count']
}).round(2)
hourly_revenue.columns = ['Total Revenue', 'Avg Revenue per Booking', 'Number of Bookings']

print("⏰ Revenue by Hour:")
display(hourly_revenue.sort_values('Total Revenue', ascending=False))

## 6. Customer Behavior Analysis

In [ ]:
# Customer type distribution
customer_dist = booked_df['customer_type'].value_counts()
customer_pct = (customer_dist / customer_dist.sum()) * 100

fig = go.Figure(data=[go.Pie(
    labels=customer_dist.index,
    values=customer_dist.values,
    hole=0.3,
    textinfo='label+percent',
    marker=dict(colors=['#2ecc71', '#3498db', '#e74c3c'])
)])

fig.update_layout(
    title='Customer Type Distribution',
    height=500
)

fig.show()

print("\n👥 Customer Types:")
for ctype, count in customer_dist.items():
    print(f"{ctype}: {count:,} bookings ({customer_pct[ctype]:.1f}%)")

In [ ]:
# Revenue by customer type
customer_revenue = booked_df.groupby('customer_type').agg({
    'actual_revenue': ['sum', 'mean', 'count']
}).round(2)
customer_revenue.columns = ['Total Revenue', 'Avg Revenue per Booking', 'Number of Bookings']
customer_revenue['% of Total Revenue'] = (
    (customer_revenue['Total Revenue'] / customer_revenue['Total Revenue'].sum()) * 100
).round(2)

print("💰 Revenue by Customer Type:")
display(customer_revenue.sort_values('Total Revenue', ascending=False))

In [ ]:
# Advance booking patterns
advance_booking = booked_df['advance_booking_days'].value_counts().sort_index()

fig = go.Figure(data=[
    go.Bar(
        x=advance_booking.index,
        y=advance_booking.values,
        marker_color='teal'
    )
])

fig.update_layout(
    title='Advance Booking Patterns',
    xaxis_title='Days in Advance',
    yaxis_title='Number of Bookings',
    height=400
)

fig.show()

print(f"\nAverage advance booking: {booked_df['advance_booking_days'].mean():.1f} days")

## 7. Operational Metrics

In [ ]:
# Cancellation and no-show rates
cancellation_rate = (booked_df['was_cancelled'].sum() / len(booked_df)) * 100
no_show_rate = (booked_df['was_no_show'].sum() / len(booked_df)) * 100
total_issues_rate = cancellation_rate + no_show_rate

print("⚠️  OPERATIONAL ISSUES")
print(f"Cancellation Rate: {cancellation_rate:.2f}%")
print(f"No-Show Rate: {no_show_rate:.2f}%")
print(f"Total Issues Rate: {total_issues_rate:.2f}%")
print(f"\nTotal Cancelled Bookings: {booked_df['was_cancelled'].sum()}")
print(f"Total No-Shows: {booked_df['was_no_show'].sum()}")

In [ ]:
# Issues by customer type
issues_by_customer = booked_df.groupby('customer_type').agg({
    'was_cancelled': ['sum', lambda x: (x.sum() / len(x)) * 100],
    'was_no_show': ['sum', lambda x: (x.sum() / len(x)) * 100]
}).round(2)
issues_by_customer.columns = ['Cancellations', 'Cancellation Rate %', 'No-Shows', 'No-Show Rate %']

print("\n📊 Issues by Customer Type:")
display(issues_by_customer)

In [ ]:
# Revenue loss breakdown
revenue_loss_breakdown = booked_df.groupby('customer_type').apply(
    lambda x: x['revenue'].sum() - x['actual_revenue'].sum()
)

fig = go.Figure(data=[
    go.Bar(
        x=revenue_loss_breakdown.index,
        y=revenue_loss_breakdown.values,
        marker_color='crimson',
        text=[f"${val:.2f}" for val in revenue_loss_breakdown.values],
        textposition='auto'
    )
])

fig.update_layout(
    title='Revenue Loss by Customer Type (Cancellations + No-Shows)',
    xaxis_title='Customer Type',
    yaxis_title='Revenue Loss ($)',
    height=400
)

fig.show()

## 8. Time Series & Trend Analysis

In [ ]:
# Daily bookings trend
daily_bookings = df[df['is_booked'] == True].groupby('date').size().reset_index(name='bookings')

# Calculate 7-day moving average
daily_bookings['ma_7'] = daily_bookings['bookings'].rolling(window=7).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=daily_bookings['date'],
    y=daily_bookings['bookings'],
    mode='lines',
    name='Daily Bookings',
    line=dict(color='lightblue', width=1),
    opacity=0.5
))

fig.add_trace(go.Scatter(
    x=daily_bookings['date'],
    y=daily_bookings['ma_7'],
    mode='lines',
    name='7-Day Moving Average',
    line=dict(color='darkblue', width=3)
))

fig.update_layout(
    title='Daily Bookings Trend with 7-Day Moving Average',
    xaxis_title='Date',
    yaxis_title='Number of Bookings',
    hovermode='x unified',
    height=500
)

fig.show()

In [ ]:
# Growth analysis - Compare first vs last month
first_month = df[df['year_month'] == df['year_month'].min()]
last_month = df[df['year_month'] == df['year_month'].max()]

first_month_bookings = first_month['is_booked'].sum()
last_month_bookings = last_month['is_booked'].sum()
booking_growth = ((last_month_bookings - first_month_bookings) / first_month_bookings) * 100

first_month_revenue = first_month[first_month['is_booked'] == True]['actual_revenue'].sum()
last_month_revenue = last_month[last_month['is_booked'] == True]['actual_revenue'].sum()
revenue_growth = ((last_month_revenue - first_month_revenue) / first_month_revenue) * 100

print("📈 GROWTH ANALYSIS (First Month vs Last Month)")
print(f"\nFirst Month: {df['year_month'].min()}")
print(f"  - Bookings: {first_month_bookings}")
print(f"  - Revenue: ${first_month_revenue:,.2f}")

print(f"\nLast Month: {df['year_month'].max()}")
print(f"  - Bookings: {last_month_bookings}")
print(f"  - Revenue: ${last_month_revenue:,.2f}")

print(f"\nGrowth:")
print(f"  - Booking Growth: {booking_growth:+.2f}%")
print(f"  - Revenue Growth: {revenue_growth:+.2f}%")

## 9. Key Insights & Recommendations

In [ ]:
# Generate automated insights
print("🎯 KEY INSIGHTS")
print("="*60)

# 1. Best performing court
best_court = court_util['Utilization %'].idxmax()
print(f"\n1. 🏆 {best_court} has the highest utilization at {court_util.loc[best_court, 'Utilization %']:.1f}%")

# 2. Peak hours
top_3_hours = hourly_util.nlargest(3, 'Utilization %')
print(f"\n2. ⏰ Peak hours are: {', '.join([f'{int(h)}:00' for h in top_3_hours.index])}")
print(f"   Average utilization during peak: {top_3_hours['Utilization %'].mean():.1f}%")

# 3. Most valuable customer segment
top_customer = customer_revenue['Total Revenue'].idxmax()
print(f"\n3. 💎 {top_customer} customers generate the most revenue: ${customer_revenue.loc[top_customer, 'Total Revenue']:,.2f}")
print(f"   ({customer_revenue.loc[top_customer, '% of Total Revenue']:.1f}% of total revenue)")

# 4. Revenue opportunity
low_util_hours = hourly_util[hourly_util['Utilization %'] < 40]
print(f"\n4. 💡 {len(low_util_hours)} hours have utilization below 40%")
print(f"   Consider promotions during: {', '.join([f'{int(h)}:00' for h in low_util_hours.index])}")

# 5. Operational concern
if total_issues_rate > 5:
    print(f"\n5. ⚠️  High cancellation/no-show rate at {total_issues_rate:.1f}%")
    print(f"   Estimated revenue loss: ${revenue_loss:,.2f}")
    highest_issue_customer = issues_by_customer['Cancellation Rate %'].idxmax()
    print(f"   {highest_issue_customer} customers have the highest cancellation rate")
else:
    print(f"\n5. ✅ Low cancellation/no-show rate at {total_issues_rate:.1f}% - excellent!")

print("\n" + "="*60)

In [ ]:
# Recommendations
print("\n📋 ACTIONABLE RECOMMENDATIONS")
print("="*60)

recommendations = []

# Pricing strategy
if hourly_revenue['Avg Revenue per Booking'].std() > 5:
    recommendations.append(
        "1. PRICING: Implement dynamic pricing - increase prices during peak hours (6-10 PM) " 
        "and offer discounts during low-demand periods"
    )

# Utilization improvement
if utilization_rate < 60:
    recommendations.append(
        "2. MARKETING: Launch targeted campaigns to fill off-peak slots (weekday mornings/afternoons). "
        "Consider corporate packages for daytime slots."
    )

# Customer retention
regular_pct = (booked_df['customer_type'] == 'Regular').sum() / len(booked_df) * 100
if regular_pct < 60:
    recommendations.append(
        f"3. RETENTION: Only {regular_pct:.1f}% are regular customers. "
        "Implement a loyalty program to convert one-time customers to regulars."
    )

# Reduce no-shows
if no_show_rate > 3:
    recommendations.append(
        "4. OPERATIONS: High no-show rate detected. Implement SMS reminders 24h before booking "
        "or require deposits for peak time slots."
    )

# Weekend strategy
if day_type_util[True] > day_type_util[False] + 20:
    recommendations.append(
        "5. CAPACITY: Weekends are significantly busier. Consider extending weekend hours "
        "or booking slots in 90-minute intervals during peak weekend times."
    )

# Court optimization
court_diff = abs(court_util.loc['Court 1', 'Utilization %'] - court_util.loc['Court 2', 'Utilization %'])
if court_diff > 10:
    recommendations.append(
        f"6. FACILITY: {court_diff:.1f}% utilization gap between courts. "
        "Investigate why one court is underperforming (location, condition, visibility)."
    )

for rec in recommendations:
    print(f"\n{rec}")

print("\n" + "="*60)

## 10. Export Analysis Results

In [ ]:
# Create summary report
summary_report = {
    'Report Generated': datetime.now().strftime('%Y-%m-%d %H:%M'),
    'Date Range': f"{df['date'].min().date()} to {df['date'].max().date()}",
    'Total Slots': total_slots,
    'Booked Slots': int(booked_slots),
    'Overall Utilization %': round(utilization_rate, 2),
    'Total Potential Revenue': round(total_potential_revenue, 2),
    'Total Actual Revenue': round(total_actual_revenue, 2),
    'Revenue Loss': round(revenue_loss, 2),
    'Cancellation Rate %': round(cancellation_rate, 2),
    'No-Show Rate %': round(no_show_rate, 2),
    'Average Advance Booking (days)': round(booked_df['advance_booking_days'].mean(), 1),
    'Court 1 Utilization %': round(court_util.loc['Court 1', 'Utilization %'], 2),
    'Court 2 Utilization %': round(court_util.loc['Court 2', 'Utilization %'], 2)
}

summary_df = pd.DataFrame([summary_report]).T
summary_df.columns = ['Value']

print("📊 EXECUTIVE SUMMARY")
display(summary_df)

# Export to CSV
summary_df.to_csv('/home/claude/futsal_summary_report.csv')
print("\n✅ Summary report exported to: futsal_summary_report.csv")

---
## 📝 Next Steps

1. **Replace mock data** with your actual futsal court data
2. **Adjust metrics** based on your specific business needs
3. **Automate this analysis** - see automation guide in README
4. **Share insights** using the Streamlit dashboard

### To Automate This Notebook:
```python
# Convert to .py script and schedule:
jupyter nbconvert --to python futsal_eda.ipynb
# Then use cron or GitHub Actions to run weekly/monthly
```